In [9]:
# Import Libraries
import yfinance as yf
import finta
import plotly.graph_objects as go
from pandas.tseries.offsets import DateOffset
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from sklearn.svm import SVC

In [10]:
# Function to Plot Bollinger Bands and Locate Intersections
def plot_bollinger_bands(stock_symbol):
    # Download historical data for the specified stock symbol
    df = yf.download(stock_symbol, start="2015-01-01", end="2023-06-07")

    # Define Bollinger Bands function
    def bollinger_bands(data, n, std, col='Close'):
        MA = data[col].rolling(window=n).mean()
        SD = data[col].rolling(window=n).std()
        data['BB_Upper'] = MA + (std * SD)
        data['BB_Lower'] = MA - (std * SD)
        return data

    # Compute Bollinger Bands with std dev of 1 and 3
    df = bollinger_bands(df, n=200, std=1)
    df.rename(columns={'BB_Upper': 'BB_Upper1', 'BB_Lower': 'BB_Lower1'}, inplace=True)
    df = bollinger_bands(df, n=200, std=2)
    df.rename(columns={'BB_Upper': 'BB_Upper2', 'BB_Lower': 'BB_Lower2'}, inplace=True)

    # Compute EMA
    df['EMA_200'] = df['Close'].ewm(span=200, adjust=False).mean()

    # Create a Candlestick chart
    trace_candle = go.Candlestick(x=df.index,
                                 open=df['Open'],
                                 high=df['High'],
                                 low=df['Low'],
                                 close=df['Close'], name='Candlestick')

    # Create traces for Bollinger Bands, SMAs, EMA
    trace_bb_upper1 = go.Scatter(x=df.index, y=df['BB_Upper1'], line={'color': 'red'}, name='BB Upper 1')
    trace_bb_lower1 = go.Scatter(x=df.index, y=df['BB_Lower1'], line={'color': 'red'}, name='BB Lower 1')
    trace_bb_upper3 = go.Scatter(x=df.index, y=df['BB_Upper2'], line={'color': 'blue'}, name='BB Upper 2')
    trace_bb_lower3 = go.Scatter(x=df.index, y=df['BB_Lower2'], line={'color': 'blue'}, name='BB Lower 2')
    trace_ema_200 = go.Scatter(x=df.index, y=df['EMA_200'], line={'color': 'yellow'}, name='EMA 200')

    # Create layout
    layout = go.Layout(title=f'{stock_symbol} Bollinger Bands and EMA 200',
                       xaxis=dict(title="Date"),
                       yaxis=dict(title="Price"),
                       width=1200,
                       height=800)

    # Create figure and add traces
    fig = go.Figure(data=[trace_candle, trace_bb_upper1, trace_bb_lower1, trace_bb_upper3, trace_bb_lower3, trace_ema_200], layout=layout)
    
    # Determine entry and exit points
    entry_points_BB = []
    exit_points_BB = []

    for i in range(1, len(df)):
        if df['Close'][i] < df['BB_Lower2'][i] and df['Close'][i-1] >= df['BB_Lower2'][i-1]:
            entry_points_BB.append((df.index[i].date(), round(df['Close'][i], 2)))
        elif df['Close'][i] > df['BB_Upper2'][i] and df['Close'][i-1] <= df['BB_Upper2'][i-1]:
            exit_points_BB.append((df.index[i].date(), round(df['Close'][i], 2)))

    # Print entry and exit points
    print("Entry Points:")
    for entry in entry_points_BB:
        print(f"Date: {entry[0]}, Price: {entry[1]}")

    print("\nExit Points:")
    for exit in exit_points_BB:
        print(f"Date: {exit[0]}, Price: {exit[1]}")

    # Show the figure
    fig.show()

In [1]:
# Function to Plot SMA and Locate Intersections
def plot_SMA(stock_symbol):
    # Download historical data for the specified stock symbol
    df = yf.download(stock_symbol, start="2015-01-01", end="2023-06-07")

    # Compute SMAs
    df['SMA_9'] = df['Close'].rolling(window=9).mean()
    df['SMA_21'] = df['Close'].rolling(window=21).mean()

    # Compute EMA
    df['EMA_200'] = df['Close'].ewm(span=200, adjust=False).mean()
    
    # Create a Candlestick chart
    trace_candle = go.Candlestick(x=df.index,
                    open=df['Open'],
                    high=df['High'],
                    low=df['Low'],
                    close=df['Close'], name='Candlestick')
    
    # Dropping unnecessary columns
    df = df.drop(columns=["Open", "High", "Low", "Adj Close", "Close", "Volume"])
    df.dropna(inplace=True)


    # Create traces for SMAs, EMA
    trace_sma_9 = go.Scatter(x=df.index, y=df['SMA_9'], line={'color': 'green'}, name='SMA 9')
    trace_sma_21 = go.Scatter(x=df.index, y=df['SMA_21'], line={'color': 'black'}, name='SMA 21')
    trace_ema_200 = go.Scatter(x=df.index, y=df['EMA_200'], line={'color': 'yellow'}, name='EMA 200')

    # Create layout
    layout = go.Layout(title= f'{stock_symbol} with SMAs and EMA 200',
                       xaxis=dict(title="Date"),
                       yaxis=dict(title="Price"),
                       width=1200,
                       height=800)

    # Create figure and add traces
    fig1 = go.Figure(data=[trace_candle, trace_sma_9, trace_sma_21, trace_ema_200], layout=layout)

    # Determine entry and exit points
    df["signal"] = 0
    
    for i in range(1, len(df)):
        if df['SMA_9'][i] > df['EMA_200'][i] and df['SMA_21'][i] > df['EMA_200'][i]:
            df.loc[i, 'signal'] = -1  # Exit signal
        elif df['SMA_9'][i] < df['EMA_200'][i] and df['SMA_21'][i] < df['EMA_200'][i]:
            df.loc[i, 'signal'] = 1  # Entry signal
    
    print(df)
    
    # Copy the new signal column to a new Series called y.
    y = df["signal"].copy()

    # Assign a copy of the sma_fast and sma_slow columns to a new DataFrame called X
    X = df[["SMA_9", "SMA_21"]].copy()
    
    # Split the Data Into Training and Testing Datasets

    # Select the start of the training period
    training_begin = X.index.min()

    # Display the training begin date
    print(training_begin)

    # Select the ending period for the training data with an offset of 3 months
    training_end = X.index.min() + DateOffset(months=24)

    # Display the training end date
    print(training_end)

    # Generate the X_train and y_train DataFrames
    X_train = X.loc[training_begin:training_end]
    y_train = y.loc[training_begin:training_end]

    # Generate the X_test and y_test DataFrames
    X_test = X.loc[training_end:]
    y_test = y.loc[training_end:]

    # Create a StandardScaler instance
    scaler = StandardScaler()

    # Apply the scaler model to fit the X-train data
    X_scaler = scaler.fit(X_train)

    # Transform the X_train and X_test DataFrames using the X_scaler
    X_train_scaled = X_scaler.transform(X_train)
    X_test_scaled = X_scaler.transform(X_test)
    
    # Create the classifier model
    svm_model = svm.SVC()

    # Fit the model to the data using X_train_scaled and y_train
    svm_model = svm_model.fit(X_train_scaled, y_train)

    # Use the trained model to predict the trading signals for the training data
    training_signal_predictions = svm_model.predict(X_train_scaled)

    # Display the sample predictions
    training_signal_predictions[:10]
    
    # Evaluate the model using a classification report
    training_report = classification_report(y_train, training_signal_predictions)
    print(training_report)
    
    # Use the trained model to predict the trading signals for the testing data.
    testing_signal_predictions = svm_model.predict(X_test_scaled)
    
    # Evaluate the model's ability to predict the trading signal for the testing data
    svm_testing_report = classification_report(y_test, testing_signal_predictions)
    print(svm_testing_report)
    
    # Create a new empty predictions DataFrame using code provided below.
    predictions_df = pd.DataFrame(index=X_test.index)
    predictions_df["predicted_signal"] = testing_signal_predictions
    predictions_df["actual_returns"] = trading_df["actual_returns"]
    predictions_df["trading_algorithm_returns"] = predictions_df["actual_returns"] * predictions_df["predicted_signal"]
    predictions_df.head()
    
    # Calculate and plot the cumulative returns for the `actual_returns` and the `trading_algorithm_returns`
    (1 + predictions_df[["actual_returns", "trading_algorithm_returns"]]).cumprod().plot()
   
    # Show the figure
    fig1.show()

In [2]:
# Function to Plot MACD and Locate Intersections
def plot_MACD(stock_symbol):
    # Download historical data for the specified stock symbol
    df = yf.download(stock_symbol, start="2015-01-01", end="2023-06-07")

    # Compute MACD
    def compute_macd(data, short=12, long=26):
        short_ema = data.ewm(span=short, adjust=False).mean()
        long_ema = data.ewm(span=long, adjust=False).mean()
        macd = short_ema - long_ema
        signal = macd.ewm(span=9, adjust=False).mean()
        return macd, signal

    # Compute MACD and MACD Signal
    macd, macd_signal = compute_macd(df['Close'])

    # Create traces for MACD, MACD Signal, and zero line
    trace_macd = go.Scatter(x=df.index, y=macd, line={'color': 'cyan'}, name='MACD')
    trace_macd_signal = go.Scatter(x=df.index, y=macd_signal, line={'color': 'magenta'}, name='MACD Signal')
    trace_zero_line = go.Scatter(x=df.index, y=[0] * len(df), line={'color': 'black', 'dash': 'dash'}, name='Zero Line')
    
    # Create layout
    layout = go.Layout(
        title="MACD Indicator",
        xaxis=dict(title="Date"),
        yaxis=dict(title="MACD/Signal"),
        width=1200,
        height=800)

    # Create figure and add traces
    fig2 = go.Figure(data=[trace_macd, trace_macd_signal, trace_zero_line], layout=layout)

    # Determine entry and exit points
    entry_points_MACD = []
    exit_points_MACD = []

    for i in range(1, len(macd)):
        if macd[i-1] < macd_signal[i-1] and macd[i] > macd_signal[i]:
            entry_points_MACD.append((df.index[i].date(), round(df['Close'][i], 2)))
        elif macd[i-1] > macd_signal[i-1] and macd[i] < macd_signal[i]:
            exit_points_MACD.append((df.index[i].date(), round(df['Close'][i], 2)))

    # Print entry and exit points
    print("Entry Points of MACD:")
    for entry in entry_points_MACD:
        print(f"Date: {entry[0]}, Price: {entry[1]}")

    print("\nExit Points of MACD:")
    for exit in exit_points_MACD:
        print(f"Date: {exit[0]}, Price: {exit[1]}")

    # Show the figure
    fig2.show()


In [3]:
# Import Data from Yahoo Finance
stocks = ['META', 'AAPL', 'AMZN', 'NFLX', 'GOOG']
dfs = {}

start_date = "2016-06-01"
end_date = "2023-06-01"

for stock in stocks:
    dfs[stock] = yf.download(stock, start=start_date, end=end_date)

# Plotting Bollinger Bands and SMAs for each stock
for stock, df in dfs.items():
    plot_bollinger_bands(stock)
    plot_SMA(stock)
    plot_MACD(stock)

NameError: name 'yf' is not defined